In [7]:
!sahi predict --source aerial/ --dataset_json_path aerial.json --model_type yolov5 --model_path models/plane_yolov5.pt --no_sliced_prediction

indexing coco dataset annotations...
Loading coco annotations: 100%|████████████████| 3/3 [00:00<00:00, 23607.71it/s]
08/17/2022 23:41:09 - INFO - numexpr.utils -   Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
08/17/2022 23:41:09 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.
Prediction time is: 134.93 ms                                                   
Prediction time is: 102.08 ms                                                   
Prediction time is: 86.36 ms                                                    
Performing inference on images: 100%|█████████████| 3/3 [00:00<00:00,  3.41it/s]
Prediction results are successfully exported to runs/predict/exp


In [65]:
import pandas as pd
import numpy as np
import numba
import re
import cv2
import ast
import matplotlib.pyplot as plt

from numba import jit

from typing import List, Union, Tuple


In [66]:
@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area

In [47]:
import pandas as pd

08/18/2022 01:38:16 - INFO - numexpr.utils -   Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
08/18/2022 01:38:16 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.


In [285]:
df1 = pd.read_csv('ground_truth.csv')
df2 = pd.read_json('16ep.json')
df11 = pd.read_csv('16o.csv')
df22 = pd.read_json('16p.json')

In [276]:
df1=df1[['bbox']]
df2=df2[['bbox']]

In [286]:
df11=df11[['bbox']]
df22=df22[['bbox']]

In [287]:
df11[['a', 'b', 'c', 'd']] = df11['bbox'].str.extractall('(\d+)').unstack().loc[:,0]

In [277]:
df1[['a', 'b', 'c', 'd']] = df1['bbox'].str.extractall('(\d+)').unstack().loc[:,0]

In [290]:
l11=[]
l22=[]
l33=[]
l44=[]
for r,row in df22.iterrows():
    l11.append(row[0][0]),
    l22.append(row[0][1]),
    l33.append(row[0][2]),
    l44.append(row[0][3])

In [293]:
result1=[]
for i in range(0,len(l1)):
               result1.append(calculate_iou(np.array([int(df11.a[i]),int(df11.b[i]),int(df11.c[i]),int(df11.d[i])]), np.array([l11[i],l22[i],l33[i],l44[i]]), form='coco'))

In [281]:
result2=[]
for i in range(0,len(l1)):
               result2.append(calculate_iou(np.array([int(df1.a[i]),int(df1.b[i]),int(df1.c[i]),int(df1.d[i])]), np.array([l11[i],l22[i],l33[i],l44[i]]), form='coco'))

In [282]:
result16_enhanced=pd.DataFrame(result2,columns=['enhanced'])

In [283]:
result16_enhanced.to_csv('iou16_enhanced.csv')

In [284]:
result16_enhanced.describe()

,enhanced
count,21.000000
mean,0.779949
std,0.060627
min,0.705395
25%,0.733457
50%,0.759524
75%,0.795557
max,0.942186


In [297]:
result16=pd.DataFrame(result1,columns=['original'])

In [298]:
result16.describe()

,original
count,21.000000
mean,0.798820
std,0.080062
min,0.637652
25%,0.765957
50%,0.792453
75%,0.842105
max,0.960000


In [299]:
!pwd

/Users/yulin/workspace/GIthub/testing/satesr/demo/sahi


In [300]:
import pandas as pd

In [9]:
df3=pd.read_json('result.json')

In [18]:
score1=df3[df3['image_id'] == 0]
score2=df3[df3['image_id'] == 1]
score3=df3[df3['image_id'] == 2]

In [19]:
score1.describe()

,image_id,score,category_id,iscrowd,area
count,21.0,21.000000,21.0,21.0,21.000000
mean,0.0,0.849911,0.0,0.0,5764.714286
std,0.0,0.039263,0.0,0.0,763.355759
min,0.0,0.736924,0.0,0.0,4189.000000
25%,0.0,0.836008,0.0,0.0,5346.000000
50%,0.0,0.864436,0.0,0.0,5776.000000
75%,0.0,0.875160,0.0,0.0,6120.000000
max,0.0,0.898247,0.0,0.0,7533.000000


In [20]:
score2.describe()

,image_id,score,category_id,iscrowd,area
count,21.0,21.000000,21.0,21.0,21.000000
mean,1.0,0.829413,0.0,0.0,5221.714286
std,0.0,0.065567,0.0,0.0,758.481453
min,1.0,0.635821,0.0,0.0,4104.000000
25%,1.0,0.804390,0.0,0.0,4425.000000
50%,1.0,0.845905,0.0,0.0,5313.000000
75%,1.0,0.869642,0.0,0.0,5680.000000
max,1.0,0.908444,0.0,0.0,6636.000000


In [21]:
score3.describe()

,image_id,score,category_id,iscrowd,area
count,21.0,21.000000,21.0,21.0,21.000000
mean,2.0,0.812491,0.0,0.0,361.857143
std,0.0,0.087477,0.0,0.0,61.389157
min,2.0,0.588235,0.0,0.0,270.000000
25%,2.0,0.796412,0.0,0.0,336.000000
50%,2.0,0.845168,0.0,0.0,374.000000
75%,2.0,0.868107,0.0,0.0,399.000000
max,2.0,0.879494,0.0,0.0,480.000000


In [301]:
re=pd.read_json('realesrgan.json')

In [303]:
re.describe()

,score,category_id,iscrowd,area
count,25.000000,25.0,25.0,25.000000
mean,0.750771,0.0,0.0,4100.000000
std,0.197500,0.0,0.0,824.497776
min,0.262448,0.0,0.0,2350.000000
25%,0.784535,0.0,0.0,3604.000000
50%,0.840800,0.0,0.0,4225.000000
75%,0.854831,0.0,0.0,4556.000000
max,0.899851,0.0,0.0,5727.000000
